# RESnet for Cifar-10

In [1]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [2]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

### define value tracker

In [3]:
def value_tracker(value_plot, value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=value,
             win = value_plot,
             update='append'
             ) 

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

### transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

#### How to Calculate mean and std in Normalize 

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
print(trainset.data.shape)

train_data_mean = trainset.data.mean( axis=(0,1,2) )
train_data_std = trainset.data.std( axis=(0,1,2) )

print(train_data_mean)
print(train_data_std)

train_data_mean = train_data_mean / 255
train_data_std = train_data_std / 255

print(train_data_mean)
print(train_data_std)

Files already downloaded and verified
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,  # original 256
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                       download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=128,  # original 256
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### make ResNet50

In [7]:
import torchvision.models.resnet as resnet 

In [8]:
resnet50 = resnet.resnet50().to(device)
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(resnet50.parameters(), lr = 0.1, momentum = 0.9, weight_decay=5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

### make plot

In [10]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))
acc_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='Accuracy', legend=['Acc'], showlegend=True))

### define acc_check function

In [11]:
def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = (100 * correct / total)
    print('Accuracy of the network on the 10000 test images: %d %%' % acc)
    if save:
        torch.save(net.state_dict(), "./model/model43_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

### Training with acc_check and model_save

In [12]:
print(len(trainloader))
epochs = 50

391


In [13]:
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    lr_sche.step()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
            print('[%d, %3d] loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    
    # Check Accuracy
    acc = acc_check(resnet50, testloader, epoch, save=1)
    value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))
    
print('Finished Training')

C:\Users\ckkim\.conda\envs\vsc\lib\site-packages\torch\optim\lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,  30] loss: 27.409385
[1,  60] loss: 5.835786
[1,  90] loss: 3.085668
[1, 120] loss: 3.009681
[1, 150] loss: 2.593116
[1, 180] loss: 2.445443
[1, 210] loss: 2.385295
[1, 240] loss: 2.318121
[1, 270] loss: 2.233349
[1, 300] loss: 2.223538
[1, 330] loss: 2.178525
[1, 360] loss: 2.136535
[1, 390] loss: 2.129084
Accuracy of the network on the 10000 test images: 21 %
[2,  30] loss: 2.090878
[2,  60] loss: 2.095297
[2,  90] loss: 2.093529
[2, 120] loss: 2.090547
[2, 150] loss: 2.057116
[2, 180] loss: 2.051410
[2, 210] loss: 2.031616
[2, 240] loss: 2.032419
[2, 270] loss: 2.030541
[2, 300] loss: 2.016779
[2, 330] loss: 1.983643
[2, 360] loss: 1.970147
[2, 390] loss: 1.960616
Accuracy of the network on the 10000 test images: 28 %
[3,  30] loss: 1.920305
[3,  60] loss: 1.911338
[3,  90] loss: 1.907330
[3, 120] loss: 1.924827
[3, 150] loss: 1.874581
[3, 180] loss: 1.885273
[3, 210] loss: 1.891991
[3, 240] loss: 1.829887
[3, 270] loss: 1.837494
[3, 300] loss: 1.815921
[3, 330] loss: 1.848081
[

[23,  30] loss: 0.666250
[23,  60] loss: 0.689345
[23,  90] loss: 0.703966
[23, 120] loss: 0.692183
[23, 150] loss: 0.697008
[23, 180] loss: 0.721590
[23, 210] loss: 0.692079
[23, 240] loss: 0.704106
[23, 270] loss: 0.732356
[23, 300] loss: 0.727052
[23, 330] loss: 0.714798
[23, 360] loss: 0.695719
[23, 390] loss: 0.737959
Accuracy of the network on the 10000 test images: 72 %
[24,  30] loss: 0.689716
[24,  60] loss: 0.667040
[24,  90] loss: 0.712589
[24, 120] loss: 0.689567
[24, 150] loss: 0.670626
[24, 180] loss: 0.671976
[24, 210] loss: 0.695286
[24, 240] loss: 0.687247
[24, 270] loss: 0.686635
[24, 300] loss: 0.669196
[24, 330] loss: 0.699553
[24, 360] loss: 0.723800
[24, 390] loss: 0.692473
Accuracy of the network on the 10000 test images: 73 %
[25,  30] loss: 0.633114
[25,  60] loss: 0.661258
[25,  90] loss: 0.665628
[25, 120] loss: 0.685356
[25, 150] loss: 0.671417
[25, 180] loss: 0.670403
[25, 210] loss: 0.674737
[25, 240] loss: 0.686046
[25, 270] loss: 0.716559
[25, 300] loss:

[44, 300] loss: 0.274154
[44, 330] loss: 0.315188
[44, 360] loss: 0.295402
[44, 390] loss: 0.314889
Accuracy of the network on the 10000 test images: 78 %
[45,  30] loss: 0.245391
[45,  60] loss: 0.245535
[45,  90] loss: 0.264300
[45, 120] loss: 0.259023
[45, 150] loss: 0.262467
[45, 180] loss: 0.272797
[45, 210] loss: 0.265561
[45, 240] loss: 0.302858
[45, 270] loss: 0.287992
[45, 300] loss: 0.296672
[45, 330] loss: 0.302868
[45, 360] loss: 0.293943
[45, 390] loss: 0.316982
Accuracy of the network on the 10000 test images: 78 %
[46,  30] loss: 0.263464
[46,  60] loss: 0.247340
[46,  90] loss: 0.248210
[46, 120] loss: 0.259375
[46, 150] loss: 0.254894
[46, 180] loss: 0.280588
[46, 210] loss: 0.273426
[46, 240] loss: 0.301519
[46, 270] loss: 0.278818
[46, 300] loss: 0.293565
[46, 330] loss: 0.270976
[46, 360] loss: 0.302437
[46, 390] loss: 0.286020
Accuracy of the network on the 10000 test images: 78 %
[47,  30] loss: 0.256410
[47,  60] loss: 0.235320
[47,  90] loss: 0.263498
[47, 120] 

![loss](images/43_ResNet_Cifar10-loss.svg)

![accuracy](images/43_ResNet_Cifar10-accuracy.svg)

### Model Accuracy Testing

In [15]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %5.2f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 79.76 %
